# Day 5: Supply Stacks

https://adventofcode.com/2022/day/5

_After the rearrangement procedure completes, what crate ends up on top of each stack?_

* Input contains a drawing of the stacks followed by move instructions
* Example drawing shows 3 stacks. Input has 9. Assume no defined limit.
* Move instructions look like

`move 1 from 2 to 1`


In [1]:
import os
from dataclasses import dataclass
from typing import Dict
from typing import List
from enum import auto
from enum import Enum


In [2]:
@dataclass
class Crate:
  name: str

  def __repr__(self):
    return f'[{self.name}]'

@dataclass
class Stack:
  name: str
  contents: List[Crate]  

@dataclass
class Move:
  count: int
  source: int
  destination: int

  def __repr__(self):
    return f'm {self.count} f {self.source} t {self.destination}'

A single line of crate information looks like
```
      [S] [C]         [Z]            
```

While reading lines of crates we don't know how many stacks there are. A subsequent line contains:
```
  [F] [J] [P]         [T]     [N]    
```

It seems fair to rely on the leading whitespace. The input actually contains fixed-length lines which could be used to deduce the total number of stacks. That seems fragile and unnecessary. It is necessary to rely on embedded whitespace however. The only way that we know `[Z]` is above `[T]` is that the crates appear at the same offset into each line.

In [3]:
def parse_crate_line(crate_line : str):
  crates : List[Crate|None] = []
  # Let's just unpack 4 characters at a time and make this easy.
  for i in range(0, len(crate_line) - 1, 4):
    a, b, c = crate_line[i:i+3]
    if (a, c) == ('[', ']'):
      crates.append(Crate(name=b))
    elif (a, c) == (' ', ' '):
      crates.append(None)
  return crates
    

_Exercise parse_crate_line()_

In [4]:
def exercise_parse_crate_line():
  lines = ['',
  '    [S] [C]         [Z]',
  '[F] [J] [P]         [T]     [N]',
  '[D] [T] [V] [M] [J] [N] [F] [M] [G]'
  ]

  for l in lines:
    print(l)
    print(parse_crate_line(l))

exercise_parse_crate_line()


[]
    [S] [C]         [Z]
[None, [S], [C], None, None, [Z]]
[F] [J] [P]         [T]     [N]
[[F], [J], [P], None, None, [T], None, [N]]
[D] [T] [V] [M] [J] [N] [F] [M] [G]
[[D], [T], [V], [M], [J], [N], [F], [M], [G]]


In [5]:
def parse_label_line(line : str):
  return [int(i.strip()) for i in line.split()]

_Exercise parse_label_line()_

In [6]:
def exercise_parse_label_line():
  for l in [' 1 2 3', '1 2   3', '    1    2   3', '1 2 3   ']:
    print(parse_label_line(l))

exercise_parse_label_line()

[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
[1, 2, 3]


In [7]:
def parse_move_line(line: str):
  parts = line.strip().split()
  if len(parts) != 6 or (parts[0], parts[2], parts[4]) != ('move', 'from', 'to'):
    raise Exception(f'Can\'t parse line {line}')
  return Move(int(parts[1]), int(parts[3]), int(parts[5]))

_Exercise parse_move_line()_

In [8]:
def exercise_parse_move_line():
  print(parse_move_line('move 5 from 10 to 20'))

exercise_parse_move_line()

m 5 f 10 t 20


In [9]:
def build_stacks_from_crates(labels : List[str], crate_rows: List[List[Crate|None]]) -> List[Stack]:
  stacks : Dict[Stack] = {label: [] for label in labels}
  for row in crate_rows:
    for stack, crate in zip(stacks.values(), row):
      if crate is not None:
        stack.append(crate)
  for stack in stacks.values():
    stack.reverse()
  return stacks

In [10]:
def load_data(filename : str) -> Dict[int, List[Crate]]:
  """ Data file lines:
    * Crates from top to bottom
    * A number under each complete stack
    * A blank line
    * Move instructions
  """
  class DataFileSection(Enum):
    CRATES = auto()
    MOVES = auto()

  section = DataFileSection.CRATES
  crate_rows : List[Crate|None] = []
  moves : List[Move] = []
  with open(filename) as f:
    for l in f.readlines():
      if len(l.strip()) == 0:
        continue
      match section:
        case DataFileSection.CRATES if l.strip()[0] == '[':
          crate_rows.append(parse_crate_line(l))
        case DataFileSection.CRATES if l.strip()[0] == '1':
          # Assume the first label is 1. Use that to confirm we're on track.
          labels = parse_label_line(l)
          section = DataFileSection.MOVES
        case DataFileSection.MOVES:
          moves.append(parse_move_line(l))
  stacks = build_stacks_from_crates(labels, crate_rows)
  return stacks, moves
  

_Exercise load_data()_

In [11]:
def get_test_filenames():
  return [os.path.join('testdata', f) for f in ['tiny.txt', 'small.txt', 'sample.txt']]

In [12]:
def exercise_load_data():
  for f in get_test_filenames():
    print(f'{f}')
    print(f'{load_data(f)}')

exercise_load_data()

testdata/tiny.txt
({1: [[H], [G]], 2: [[F]]}, [m 1 f 1 t 2, m 1 f 2 t 1, m 1 f 1 t 2])
testdata/small.txt
({1: [[W], [D], [R], [V], [G]], 2: [[X], [T], [B], [V], [H], [S]], 4: [[Y], [M], [N]], 5: [[Z], [J], [N], [F]]}, [m 3 f 1 t 4, m 1 f 4 t 2, m 4 f 4 t 5, m 2 f 5 t 1])
testdata/sample.txt
({1: [[D], [L], [J], [R], [V], [G], [F]], 2: [[T], [P], [M], [B], [V], [H], [J], [S]], 3: [[V], [H], [M], [F], [D], [G], [P], [C]], 4: [[M], [D], [P], [N], [G], [Q]], 5: [[J], [L], [H], [N], [F]], 6: [[N], [F], [V], [Q], [D], [G], [T], [Z]], 7: [[F], [D], [B], [L]], 8: [[M], [J], [B], [S], [V], [D], [N]], 9: [[G], [L], [D]]}, [m 3 f 4 t 6, m 1 f 5 t 8, m 3 f 7 t 3, m 4 f 5 t 7, m 1 f 7 t 8, m 3 f 9 t 4, m 2 f 8 t 2, m 4 f 4 t 5, m 2 f 5 t 1])


In [13]:
def perform_moves(stacks: Dict[int, List[Crate]], moves: List[Move]):
  for move in moves:
    crates = stacks[move.source][-move.count:]
    crates.reverse()
    stacks[move.destination] += crates
    del stacks[move.source][-move.count:]


In [14]:
def exercise_perform_moves():
    def starting_stacks():
        return {
            1 : [Crate('m'), Crate('n'), Crate('o')],
            2 : [Crate('a'), Crate('b'), Crate('c')]
        }

    all_moves = [
        [Move(1, 1, 2)],
        [Move(3, 1, 2), Move(1, 2, 1)],
        [Move(3, 2, 1), Move(1, 1, 2)],
        [Move(1, 2, 1), Move(1, 1, 2)],
        [Move(3, 2, 1), Move(3, 1, 2)],
        [Move(3, 2, 1), Move(3, 1, 2), Move(1, 2, 1), Move(2, 1, 2), Move(1, 2, 1)]
    ]
    for moves in all_moves:
        stacks = starting_stacks()
        perform_moves(stacks, moves)
        print(stacks)

exercise_perform_moves()

{1: [[m], [n]], 2: [[a], [b], [c], [o]]}
{1: [[m]], 2: [[a], [b], [c], [o], [n]]}
{1: [[m], [n], [o], [c], [b]], 2: [[a]]}
{1: [[m], [n], [o]], 2: [[a], [b], [c]]}
{1: [[m], [n], [o]], 2: [[a], [b], [c]]}
{1: [[m], [n], [o]], 2: [[a], [b], [c]]}


In [15]:
def stack_tops(stacks):
    letter = lambda x : '0' if len(x) == 0 else x[-1].name
    return ''.join([letter(s) for s in stacks.values()])

In [16]:
def solver(filename = 'input.txt'):
    stacks, moves = load_data(filename)
    perform_moves(stacks, moves)
    return stack_tops(stacks)

In [17]:
def exercise_solver():
  for f in get_test_filenames():
    print(f'{f}')
    stacks, moves = load_data(f)
    print(f'start {stacks}')
    perform_moves(stacks, moves)
    print(f'end {stacks}')
    print(f' {solver(f)}')

exercise_solver()


testdata/tiny.txt
start {1: [[H], [G]], 2: [[F]]}
end {1: [[H]], 2: [[F], [G]]}
 HG
testdata/small.txt
start {1: [[W], [D], [R], [V], [G]], 2: [[X], [T], [B], [V], [H], [S]], 4: [[Y], [M], [N]], 5: [[Z], [J], [N], [F]]}
end {1: [[W], [D], [M], [N]], 2: [[X], [T], [B], [V], [H], [S], [R]], 4: [[Y]], 5: [[Z], [J], [N], [F], [V], [G]]}
 NRYG
testdata/sample.txt
start {1: [[D], [L], [J], [R], [V], [G], [F]], 2: [[T], [P], [M], [B], [V], [H], [J], [S]], 3: [[V], [H], [M], [F], [D], [G], [P], [C]], 4: [[M], [D], [P], [N], [G], [Q]], 5: [[J], [L], [H], [N], [F]], 6: [[N], [F], [V], [Q], [D], [G], [T], [Z]], 7: [[F], [D], [B], [L]], 8: [[M], [J], [B], [S], [V], [D], [N]], 9: [[G], [L], [D]]}
end {1: [[D], [L], [J], [R], [V], [G], [F], [P], [D]], 2: [[T], [P], [M], [B], [V], [H], [J], [S], [J], [F]], 3: [[V], [H], [M], [F], [D], [G], [P], [C], [L], [B], [D]], 4: [[M], [D]], 5: [[G], [L]], 6: [[N], [F], [V], [Q], [D], [G], [T], [Z], [Q], [G], [N]], 7: [[F], [N], [H], [L]], 8: [[M], [J], [B], [S]

# Solution

In [18]:
solver()

'QMBMJDFTD'